 
1. [Regresión Logística](#id1)
2. [Random Forest](#id2)



In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import confusion_matrix
from string import punctuation
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Sacamos el dataframe final

In [2]:
df = pd.read_csv('../data/processed/Reviews_FINAL.csv',index_col=0)
df.dropna(inplace=True)

In [3]:
lista_target = []
for i in df['Estrellas']:
    if i < 4: 
        lista_target.append(0)
    else:
        lista_target.append(1)

df['Target'] = lista_target

In [4]:
df

,Estrellas,Reseñas,Sentimiento,Texto procesado,Target
0,5,Fiable con servicio al comprador incomparable....,1,fiabl servici comprador incompar total recomen...,1
1,1,Amazon exige que le entregue la firma del vend...,0,amazon exig entreg firm vendedor entreg produc...,0
2,1,Hace unos días compré un lote y la…Hace unos d...,0,hac dias compr lot la…hac dias compr lot verd ...,0
3,5,Amazon muy buena experiencia Años haciendo ped...,1,amazon buen experient años hac ped siend prim ...,1
4,1,La peor tienda online Es un completo asco de t...,0,peor tiend onlin complet asco tiend onlin hac ...,0
...,...,...,...,...,...
56695,1,"Viagogo , estafa pura y dura Compré unas entra...",0,viagog estaf pur dur compr unas entrad manuel ...,0
56696,1,MALDITOS ESTAFADORESMALDITOS ESTAFADORESMe ha ...,0,maldit estafadoresmaldit estafadoresm pas exac...,0
56697,1,VIAGOGO es una estafa absoluta VIAGOGO es una ...,0,viagog estaf absolut viagog estaf absolut pag ...,0
56698,1,TIMO con mayúsculas !!!TIMO con mayúsculas !!!...,0,tim mayuscul tim mayuscul import dar baj tarje...,0


## Regresión Logística <a name="id1"></a>


#### Probamos con CountVectorizer

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['Texto procesado'], df['Target'], test_size=0.2)

vectorizer = CountVectorizer(ngram_range=(1,3))
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

classifier = LogisticRegression(C=1)
classifier.fit(X_train_vectors, y_train)

y_pred = classifier.predict(X_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

confusion_matrix(y_test, y_pred)

Precisión del modelo: 0.9475216087493386


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[5082,  366],
       [ 229, 5661]], dtype=int64)

MemoryError: Unable to allocate 560. GiB for an array with shape (45352, 1658522) and data type int64

#### Probamos con TfIdfVectorizer

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['Texto procesado'], df['Target'], test_size=0.2)

vectorizer = TfidfVectorizer(ngram_range=(1,3))
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

classifier = LogisticRegression(C=0.1)
classifier.fit(X_train_vectors, y_train)



LogisticRegression(C=0.1)

In [16]:
puntajes_tfidf = X_test_vectors.toarray()[0]
vocabulario = vectorizer.vocabulary_
vocabulario

for palabra, puntaje in zip(vocabulario, puntajes_tfidf):
    if float(puntaje) > 0:
        print(palabra, ":", puntaje)

MemoryError: Unable to allocate 140. GiB for an array with shape (11338, 1652191) and data type float64

In [10]:
y_pred = classifier.predict(X_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

confusion_matrix(y_test, y_pred)

Precisión del modelo: 0.9461986240959604


array([[5006,  387],
       [ 223, 5722]], dtype=int64)

#### Comprobación del modelo

----

In [54]:
df_predicciones = pd.DataFrame()
df_predicciones['Real'] = y_test
df_predicciones['Predicción'] = y_pred
df_predicciones['Frase'] = X_test

In [55]:
df_predicciones[df_predicciones['Real']-df_predicciones['Predicción']>3]

,Real,Predicción,Frase


In [56]:
df_comprobacion  = df_predicciones[df_predicciones['Real']-df_predicciones['Predicción']!= 0]
df_comprobacion.iloc[0,-1]

'sugerent dia siguient entreg reserv men 24 hor si gustari asient pong junt aleatori especial si espaci avion graci'

In [57]:
df_predicciones['Predicción'].value_counts()

1    6415
0    5636
Name: Predicción, dtype: int64

In [58]:
from nltk.stem.snowball import SnowballStemmer

stopwords = nltk.corpus.stopwords.words('spanish')
stemmer = SnowballStemmer("spanish")
def preprocess_text(text):
    # Eliminar puntuación
    text = ''.join([c for c in text if c not in punctuation])

    # Convertir a minúsculas
    text = text.lower()

    # Tokenización
    tokens = nltk.word_tokenize(text)

    # Eliminar palabras de parada
    tokens = [word for word in tokens if word not in stopwords]

    #Aplicar un stemmer
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_text = " ".join(stemmed_tokens)
    return stemmed_text

In [63]:
new_text = "Patético"
processed_text = preprocess_text(new_text)
new_text_vector = vectorizer.transform([processed_text])
prediction = classifier.predict(new_text_vector)
prediction


array([0], dtype=int64)

In [60]:
print(new_text)
print(processed_text)
print(new_text_vector)

puntajes_tfidf = new_text_vector.toarray()[0]
print(puntajes_tfidf)
vocabulario = vectorizer.vocabulary_



No me ha gustado nada
gust
  (0, 22542)	1.0
[0. 0. 0. ... 0. 0. 0.]


## Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

## Grid Search CV <a name="id2"></a>

Vamos a probar con un Grid Search CV

In [23]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['Texto procesado'], df['Target'], test_size=0.2)

vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

pipe = Pipeline(steps=[("classifier", RandomForestClassifier())])

logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': [0.01,0.1,0.5,1,5]
}

random_forest_params = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [75,80,96,120],
    'classifier__max_features': [7,9,10],
    'classifier__max_depth': [4,5,6]
} #Esta es la combinación que mayor score nos da

search_space = [random_forest_params, logistic_params]

grid = GridSearchCV(pipe,
                   search_space,
                   cv = 10,
                   n_jobs=-1)

grid.fit(X_train_vectors, y_train)

c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
50 fits failed out of a total of 460.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\s

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [RandomForestClassifier()],
                          'classifier__max_depth': [4, 5, 6],
                          'classifier__max_features': [7, 9, 10],
                          'classifier__n_estimators': [75, 80, 96, 120]},
                         {'classifier': [LogisticRegression(C=5)],
                          'classifier__C': [0.01, 0.1, 0.5, 1, 5],
                          'classifier__penalty': ['l1', 'l2']}])

In [25]:
print("Best estimator:", grid.best_estimator_)
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

Best estimator: Pipeline(steps=[('classifier', LogisticRegression(C=5))])
Best params: {'classifier': LogisticRegression(C=5), 'classifier__C': 5, 'classifier__penalty': 'l2'}
Best score: 0.9346938775510205


In [26]:
y_pred = grid.predict(X_test_vectors)
confusion_matrix(y_test, y_pred)

array([[6477,  377],
       [ 470, 6152]], dtype=int64)

----

## XGBoost

In [27]:
import xgboost

In [28]:
X_train, X_val, y_train, y_val = train_test_split(df['Texto procesado'], df['Target'], test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_val)

xgb = xgboost.XGBClassifier(n_estimators = 110,max_depth = 4, n_jobs = -1, early_stopping_rounds = 18)
xgb.fit(X_train_vectors,y_train, eval_set = [(X_test_vectors, y_val)])



[0]	validation_0-logloss:0.62059
[1]	validation_0-logloss:0.57600
[2]	validation_0-logloss:0.54442
[3]	validation_0-logloss:0.51651
[4]	validation_0-logloss:0.49610
[5]	validation_0-logloss:0.47737
[6]	validation_0-logloss:0.46165
[7]	validation_0-logloss:0.44651
[8]	validation_0-logloss:0.43574
[9]	validation_0-logloss:0.42633
[10]	validation_0-logloss:0.41728
[11]	validation_0-logloss:0.40865
[12]	validation_0-logloss:0.40078
[13]	validation_0-logloss:0.39445
[14]	validation_0-logloss:0.38807
[15]	validation_0-logloss:0.38246
[16]	validation_0-logloss:0.37702
[17]	validation_0-logloss:0.37194
[18]	validation_0-logloss:0.36734
[19]	validation_0-logloss:0.36266
[20]	validation_0-logloss:0.35870
[21]	validation_0-logloss:0.35520
[22]	validation_0-logloss:0.35176
[23]	validation_0-logloss:0.34787
[24]	validation_0-logloss:0.34484
[25]	validation_0-logloss:0.34135
[26]	validation_0-logloss:0.33877
[27]	validation_0-logloss:0.33590
[28]	validation_0-logloss:0.33330
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=18,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=110, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [29]:
pred = xgb.predict(X_test_vectors)
pred_train = xgb.predict(X_train_vectors)
xgb.score(X_test_vectors, y_val)

0.892104482042149

In [30]:
confusion_matrix(y_val, pred)

array([[6531,  281],
       [1173, 5491]], dtype=int64)